# L'analyse de sentiments
Maintenant que nous avons vu les vecteurs de mots, nous pouvons commencer à étudier l'analyse des sentiments. L'objectif est de trouver des points communs entre les documents, étant entendu que des vecteurs *combinés* de manière similaire devraient correspondre à des sentiments similaires.

Bien que le champ d'application de l'analyse des sentiments soit très large, nous concentrerons notre travail de deux manières.

### 1. Classification de la polarité
Nous n'essaierons pas de déterminer si une phrase est objective ou subjective, s'il s'agit d'un fait ou d'une opinion. Nous nous contenterons de déterminer si le texte exprime une opinion *positive*, *négative* ou *neutre*.
### 2. Champ d'application au niveau du document
Nous essayons également d'agréger toutes les phrases d'un document ou d'un paragraphe pour obtenir une opinion globale.
### 3. Analyse grossière
Nous n'essaierons pas d'effectuer une analyse fine qui déterminerait le degré de positivité/négativité. En d'autres termes, nous n'essayons pas de deviner le nombre d'étoiles attribuées par un évaluateur, mais simplement de déterminer si l'évaluation est positive ou négative.

## Grandes étapes :
* Tout d'abord, il faut prendre en compte le texte à analyser. Un modèle formé à l'analyse de longs paragraphes de critiques de films peut ne pas être efficace sur des tweets. Veillez à utiliser un modèle adapté à la tâche à accomplir.
* Ensuite, décidez du type d'analyse à effectuer. Dans la section précédente sur la classification des textes, nous avons utilisé une technique de sac de mots qui ne prenait en compte que les tokens uniques, ou *unigrammes*. Certains modèles rudimentaires d'analyse des sentiments vont plus loin et prennent en compte les combinaisons de deux mots, ou *bigrammes*. Dans cette section, nous aimerions travailler avec des phrases complètes, et pour cela nous allons importer un lexique NLTK entraîné appelé *VADER*.

## Le module VADER de NLTK
VADER est un module NLTK qui fournit des scores de sentiment basés sur les mots utilisés ("complètement" augmente un score, tandis que "légèrement" le réduit), sur la majuscule et la ponctuation ("GREAT !!!" est plus fort que "great."), et sur les négations (des mots comme "isn't" et "doesn't" affectent le résultat).
<br>Pour consulter le code source, visitez le site https://www.nltk.org/_modules/nltk/sentiment/vader.html

**Téléchargez le lexique VADER **Vous ne devez le faire qu'une seule fois.

In [1]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

La fonction `SentimentIntensityAnalyzer()` de VADER prend une chaîne et renvoie un dictionnaire de scores dans chacune des quatre catégories :

* negative
* neutral
* positive
* compound *(calculé en normalisant les scores ci-dessus)*

In [3]:
a = 'This was a good movie.'
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [4]:
a = 'This was the best, most awesome movie EVER MADE!!!'
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877}

In [5]:
a = 'This was the worst film to ever disgrace the screen.'
sid.polarity_scores(a)

{'neg': 0.477, 'neu': 0.523, 'pos': 0.0, 'compound': -0.8074}

## Utiliser VADER pour analyser les critiques Amazon
Pour cet exercice, nous allons appliquer `SentimentIntensityAnalyzer` à un ensemble de données de 10 000 critiques Amazon. Comme nos ensembles de données de critiques de films, ils sont étiquetés comme "pos" ou "neg". A la fin, nous déterminerons la précision de notre analyse de sentiment avec VADER.

In [6]:
import numpy as np
import pandas as pd

df = pd.read_csv('./amazonreviews.tsv', sep='\t')
df.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [7]:
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

### Nettoyer les données (optionnel) :
Rappelons que notre fichier moviereviews.tsv contenait des enregistrements vides. Vérifions s'il en existe dans amazonreviews.tsv.

In [8]:
# REMOVE NaN VALUES AND EMPTY STRINGS:
df.dropna(inplace=True)

blanks = []  # start with an empty list

for i,lb,rv in df.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df.drop(blanks, inplace=True)

In [9]:
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

Dans ce cas, il n'y avait pas d'enregistrements vides. C'est bien !

## Examinons le premier review avec VADER

In [10]:
sid.polarity_scores(df.loc[0]['review'])

{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'compound': 0.9454}

In [11]:
df.loc[0]['label']

'pos'

C'est formidable ! Notre premier avis a été qualifié de "positif" et a obtenu une note composée positive.

## Ajouter des scores et des étiquettes au DataFrame
Dans la section suivante, nous allons ajouter des colonnes au DataFrame original pour stocker les dictionnaires polarity_score, les scores composés extraits et les nouvelles étiquettes "pos/neg" dérivées du score composé. Nous utiliserons cette dernière colonne pour effectuer un test de précision.

In [12]:
df['scores'] = df['review'].apply(lambda review: sid.polarity_scores(review))

df.head()

,label,review,scores
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co..."
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co..."
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com..."
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com..."
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp..."


In [13]:
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])

df.head()

,label,review,scores,compound
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781


In [14]:
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

df.head()

,label,review,scores,compound,comp_score
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos


## Rapport sur la précision
Enfin, nous utiliserons scikit-learn pour déterminer dans quelle mesure VADER s'est rapproché de nos 10 000 étiquettes d'origine.

In [15]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [16]:
accuracy_score(df['label'],df['comp_score'])

0.7091

In [17]:
print(classification_report(df['label'],df['comp_score']))

              precision    recall  f1-score   support

         neg       0.86      0.51      0.64      5097
         pos       0.64      0.91      0.75      4903

   micro avg       0.71      0.71      0.71     10000
   macro avg       0.75      0.71      0.70     10000
weighted avg       0.75      0.71      0.70     10000



In [18]:
print(confusion_matrix(df['label'],df['comp_score']))

[[2623 2474]
 [ 435 4468]]


Cela nous indique que VADER a correctement identifié un avis Amazon comme étant "positif" ou "négatif" dans environ 71% des cas.
## Suivant : Projet d'analyse des sentiments